In [ ]:
import os
import datetime
from PIL import Image
import PIL.ExifTags

In [2]:
def getOriginalDate(file):
    try:
        with Image.open(file) as img:
            exif = {
                PIL.ExifTags.TAGS[k]: v
                for k, v in img._getexif().items()
                if k in PIL.ExifTags.TAGS
            }
            #print(exif['DateTimeOriginal'])

        temp = exif['DateTimeOriginal'].split(' ')
        date = temp[0].split(':')
        time = temp[1].split(':')
        return datetime.datetime(int(date[0]),int(date[1]),int(date[2]),int(time[0]),int(time[1]),int(time[2])).strftime("%Y_%m_%d_%H_%M_%S")
    except:
        return datetime.datetime.fromtimestamp(os.path.getmtime(file)).strftime("%Y_%m_%d_%H_%M_%S")

In [3]:
# Rename files
def renameFiles(files, folder_path, logfile):
    for file in files:
        if os.path.isdir(os.path.join(folder_path,file)):
            print('Entering Subdirectory: ' + os.path.join(folder_path, file))
            renameFiles(os.listdir(os.path.join(folder_path, file)),os.path.join(folder_path, file),logfile)
            continue

        # Get file extension
        tmp = file.split('.')
        if len(tmp)>1:
            file_extension = "."+tmp[1]
        else:
            file_extension = ""

        # Get date formatted name for file
        try:
            new_name = getOriginalDate(os.path.join(folder_path,file))
        except:
            logfile.write(f"!!!!! FAILED TO RENAME FILE @ {os.path.join(folder_path, file)} - ISSUE GETTING DATE !!!!!\n")
            print(f"FAILED TO RENAME {os.path.join(folder_path, file)} - ISSUE GETTING DATE\n")
            continue

        # Combine name with file extension
        new_file = new_name+file_extension

        # If the name isn't changing, skip
        if file == new_file:
            logfile.write(f"Skipping {os.path.join(folder_path, file)}\n")
            continue

        # Handle FileExistsError
        copy_no = 1
        while os.path.exists(os.path.join(folder_path,new_file)):
            new_file = new_name+f' ({copy_no})'+file_extension
            copy_no += 1

        # Rename file
        try:
            os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_file))
        except:
            logfile.write(f"!!!!! FAILED TO RENAME FILE @ {os.path.join(folder_path, file)} - RENAME OPERATION FAILED !!!!!\n")
            print(f"FAILED TO RENAME {os.path.join(folder_path, file)} - RENAME OPERATION FAILED\n")
            continue

        # Log file name change
        logfile.write(os.path.join(folder_path,file)+"  ---->  " +os.path.join(folder_path,new_file)+"\n")


In [ ]:
FOLDER_PATH = r'E:\Pictures\Photo Collection\Cars' # Get path to files 
files = os.listdir(FOLDER_PATH) # Get files 

LOG_FOLDER = 'Logs'
if not os.path.exists(LOG_FOLDER):
    os.mkdir(LOG_FOLDER)

# Log files and corresponding modification dates
log_file_name = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+".txt"

with open(os.path.join(LOG_FOLDER, log_file_name),"a") as logfile:
    renameFiles(files,FOLDER_PATH,logfile)
                
